In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
import pandas as pd
Data =  pd.read_json('/content/drive/My Drive/Sarcasm Detection/Arabic News Headlines Dataset.json', orient = 'split')
Data.head()

,headlines,topic,length,is_sarcastic
0,طالب يصبح متعلّماً بعد دراسته وحدة عن القائد ف...,Society,12,1
1,سموحة يقلص الفارق ويسجل هدفا ثانيا في شباك الم...,Sports,11,0
2,ليتها الحدود – الولايات المتحدة تهدد بفرض عقوب...,Misc,17,1
3,وزيرة الهجرة تتواصل مع سفير مصر في نيوزيلندا ل...,None,14,0
4,ليتها الحدود – مصر تقرر إرسال أعضاء السائح الب...,Misc,12,1


In [0]:
X = Data['headlines']
Labels = Data['is_sarcastic']

In [0]:
import numpy as np
max_sequence_length = np.max(Data['length'])
print("max length is:", max_sequence_length)

max length is: 34


In [0]:
import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

word_index = tokenizer.word_index   # a dictionary of each word and its index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 20595 unique tokens.


In [0]:
from keras.preprocessing.sequence import pad_sequences
data = pad_sequences(sequences, maxlen=max_sequence_length)
print('Shape of data tensor:', data.shape)

Shape of data tensor: (6669, 34)


In [0]:
from keras.utils import to_categorical
labels = to_categorical(np.asarray(Labels))  ## one hot of the output
print('Shape of label tensor:', labels.shape)

Shape of label tensor: (6669, 2)


In [0]:
import pickle as p
fastText = p.load(open('/content/drive/My Drive/Sarcasm Detection/FastText_Wiki_Embeddings.p', 'rb'))

In [0]:
# prepare embedding matrix  (matrix of embeddings of the words in our dataset)
num_words = len(word_index) + 1
# initialize the np array
unKnown = list()
embedding_matrix = np.zeros((num_words, 300))
l=0
# find the embeddings from the pre trained embedding matrix of fast text wiki
for word, i in word_index.items():
    embedding_vector = fastText.get(word)
    if embedding_vector is not None:
        l+=1
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
      unKnown.append(word)
print("the number of words found in fast text embeddings =", l)
print("the number of words not found in fast text embeddings =", 20595- l)

the number of words found in fast text embeddings = 17289
the number of words not found in fast text embeddings = 3306


In [0]:
from keras.initializers import Constant
from keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling2D, Dense, Dropout, LSTM
from keras.models import Model

In [0]:
# shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]


## split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [0]:
embedding_layer = Embedding(num_words,
                            300,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=max_sequence_length,
                            trainable=False)

In [0]:
# the model

sequence_input = Input(shape=(max_sequence_length,), dtype='int32')             ## input layer
embedded_sequences = embedding_layer(sequence_input)                            ## embeddings layer
X = Conv1D(300, 3, activation='relu')(embedded_sequences)                       ## CNN layer
X = Conv1D(300, 3, activation='relu')(X)                                        ## CNN layer
X = LSTM(300, return_sequences=True)(X)                                         ## LSTM layer
X = Dropout(0.5)(X)                                                             ## Dropout layer
X = LSTM(300, return_sequences=False)(X)                                        ## LSTM layer
X = Dropout(0.5)(X)                                                             ## Dropout layer
X = Dense(300, activation = "sigmoid")(X)                                       ## Fully Connected
preds = Dense(2, activation='softmax')(X)                                       ## softmax layer

model = Model(sequence_input, preds)

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [0]:
from keras import backend as K

def recall(y_true, y_pred):
        
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall


      
def precision(y_true, y_pred):
        
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
      

def f1(y_true, y_pred):
    
    Precision = precision(y_true, y_pred)
    Recall = recall(y_true, y_pred)
    f1 = 2*((Precision*Recall)/(Precision+Recall+K.epsilon()))
    
    return f1

In [0]:
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.3, amsgrad=False)

In [0]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = adam,
              metrics = ['acc'])

model.fit(x_train, y_train, 
          validation_split = 0.2, 
          batch_size = 32, 
          epochs = 50)

loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', accuracy)

Instructions for updating:
Use tf.cast instead.
Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 16s 4ms/step - loss: 0.4948 - acc: 0.7436 - val_loss: 0.3425 - val_acc: 0.8482
Epoch 2/50
4001/4001 [==============================] - 12s 3ms/step - loss: 0.3128 - acc: 0.8605 - val_loss: 0.3014 - val_acc: 0.8761
Epoch 3/50
4001/4001 [==============================] - 12s 3ms/step - loss: 0.2823 - acc: 0.8788 - val_loss: 0.2837 - val_acc: 0.8841
Epoch 4/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.2655 - acc: 0.8920 - val_loss: 0.2728 - val_acc: 0.8851
Epoch 5/50
4001/4001 [==============================] - 12s 3ms/step - loss: 0.2524 - acc: 0.8963 - val_loss: 0.2669 - val_acc: 0.8861
Epoch 6/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.2434 - acc: 0.8983 - val_loss: 0.2627 - val_acc: 0.8821
Epoch 7/50
4001/4001 [==============================] - 12s 3ms/step - loss: 0.2368 - acc: 0.9

In [0]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = adam,
              metrics = [precision])
model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 50)

loss, prec = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', prec)


Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.4684 - precision: 0.7546 - val_loss: 0.3337 - val_precision: 0.8541
Epoch 2/50
4001/4001 [==============================] - 12s 3ms/step - loss: 0.3074 - precision: 0.8643 - val_loss: 0.2978 - val_precision: 0.8741
Epoch 3/50
4001/4001 [==============================] - 12s 3ms/step - loss: 0.2763 - precision: 0.8808 - val_loss: 0.2837 - val_precision: 0.8791
Epoch 4/50
4001/4001 [==============================] - 12s 3ms/step - loss: 0.2580 - precision: 0.8938 - val_loss: 0.2740 - val_precision: 0.8831
Epoch 5/50
4001/4001 [==============================] - 12s 3ms/step - loss: 0.2450 - precision: 0.8988 - val_loss: 0.2677 - val_precision: 0.8821
Epoch 6/50
4001/4001 [==============================] - 12s 3ms/step - loss: 0.2391 - precision: 0.9025 - val_loss: 0.2620 - val_precision: 0.8841
Epoch 7/50
4001/4001 [==============================] - 13s 3ms/step -

In [32]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = adam,
              metrics = [recall])

model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 50)

loss, rec = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', rec)

Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 16s 4ms/step - loss: 0.4925 - recall: 0.7571 - val_loss: 0.3691 - val_recall: 0.8332
Epoch 2/50
4001/4001 [==============================] - 12s 3ms/step - loss: 0.3261 - recall: 0.8510 - val_loss: 0.3265 - val_recall: 0.8521
Epoch 3/50
4001/4001 [==============================] - 12s 3ms/step - loss: 0.2910 - recall: 0.8743 - val_loss: 0.3098 - val_recall: 0.8631
Epoch 4/50
4001/4001 [==============================] - 12s 3ms/step - loss: 0.2722 - recall: 0.8840 - val_loss: 0.2999 - val_recall: 0.8661
Epoch 5/50
4001/4001 [==============================] - 12s 3ms/step - loss: 0.2627 - recall: 0.8923 - val_loss: 0.2952 - val_recall: 0.8691
Epoch 6/50
4001/4001 [==============================] - 12s 3ms/step - loss: 0.2532 - recall: 0.8943 - val_loss: 0.2921 - val_recall: 0.8701
Epoch 7/50
4001/4001 [==============================] - 12s 3ms/step - loss: 0.2471 - recall: 0.8975 - val

In [29]:
model.compile(loss = 'categorical_crossentropy',
          optimizer = adam, 
          metrics = [f1])

model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 50)

loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test f1 score is: ', accuracy)

Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.4322 - f1: 0.7891 - val_loss: 0.3247 - val_f1: 0.8511
Epoch 2/50
4001/4001 [==============================] - 12s 3ms/step - loss: 0.3028 - f1: 0.8688 - val_loss: 0.2920 - val_f1: 0.8701
Epoch 3/50
4001/4001 [==============================] - 12s 3ms/step - loss: 0.2743 - f1: 0.8855 - val_loss: 0.2791 - val_f1: 0.8761
Epoch 4/50
4001/4001 [==============================] - 12s 3ms/step - loss: 0.2570 - f1: 0.8938 - val_loss: 0.2718 - val_f1: 0.8821
Epoch 5/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.2471 - f1: 0.9008 - val_loss: 0.2645 - val_f1: 0.8851
Epoch 6/50
4001/4001 [==============================] - 12s 3ms/step - loss: 0.2406 - f1: 0.9025 - val_loss: 0.2608 - val_f1: 0.8851
Epoch 7/50
4001/4001 [==============================] - 12s 3ms/step - loss: 0.2355 - f1: 0.9048 - val_loss: 0.2585 - val_f1: 0.8891
Epoch 8/50
4001/4001 

In [0]:
 model.save_weights('/content/drive/My Drive/Sarcasm Detection/Plots/CNN_LSTM_optimized_Weights.h5')